In [1]:
from skimage import segmentation
from skimage.future import graph
from matplotlib import image
from skimage.color import label2rgb
import cv2
import os
import numpy as np
def _weight_mean_color(graph, src, dst, n):
  diff = graph.nodes[dst]['mean color'] - graph.nodes[n]['mean color']
  diff = np.linalg.norm(diff)
  return {'weight': diff}
def merge_mean_color(graph, src, dst):
  graph.nodes[dst]['total color'] += graph.nodes[src]['total color']
  graph.nodes[dst]['pixel count'] += graph.nodes[src]['pixel count']
  graph.nodes[dst]['mean color'] = (graph.nodes[dst]['total color'] /
  graph.nodes[dst]['pixel count'])

In [2]:
def segment_image(img, compactness, thresh, n_segments=200):
  labels = segmentation.slic(img, compactness=compactness, n_segments=n_segments)
  g = graph.rag_mean_color(img, labels)
  labels2 = graph.merge_hierarchical(labels, g, thresh=thresh,
    rag_copy=False,
    in_place_merge=True,
    merge_func=merge_mean_color,
    weight_func=_weight_mean_color)
  
  h, w = img.shape[0], img.shape[1]
  fground_label = labels2[h // 2, w // 2]

  return get_foreground(labels2, foreground_value=fground_label)
  
def get_foreground(segmented_image, foreground_value = 255):
  foreground_pixels = []

  indices = np.where(segmented_image == foreground_value)
  return np.stack((indices[1], indices[0]), 1)

def get_prediction(dataset): # dataset is a list of images
  prediction = []

  # for each image in the dataset
  for image in dataset:
    foreground_pixels = segment_image(image)
    # add those to your list of predictions
    prediction.append(foreground_pixels)
  
  return prediction

In [8]:
from tqdm import tqdm

comp = 60
thresh = 80
image_path = './data/test/masks/'
filenames = [image_path + name for name in os.listdir(image_path)]
target_results = []

for path in tqdm(filenames):
  img = cv2.imread(path, 0)
  fg = get_foreground(img, foreground_value=1)
  target_results.append(fg)

100%|██████████| 1832/1832 [00:21<00:00, 83.97it/s]


In [9]:
np.save('test_targets.npy', target_results)

c:\VSCodeDrive\image_segmentation\env\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)
